In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from pydantic import BaseModel, Field
from typing import Dict, List
from langchain_community.tools import DuckDuckGoSearchResults

In [3]:
# 상태 정보 정의

class State(BaseModel):
    company_name: str = Field(description="회사 이름")
    segment: Dict[str, dict] = Field(description="사업부 정보", default_factory=dict)
    PER: float = Field(description="기업의 현재 PER(TTM)", default=0)
    peer_list: List[str] = Field(description="경쟁사 리스트", default_factory=list)
    average_peer_PER: float = Field(description="경쟁사 PER 평균", default=0)


In [4]:
# 상태 정보 초기화

state = State(company_name="삼성전자")

In [5]:
# 데이터 로드 TODO: 사업보고서 혹은 애널리스트 리포트 retrieve해서 데이터 로드
import pandas as pd

naver_income_stmt_cum = pd.read_csv("samsung_segment_sales+income_stmt_cum.csv",header=0)
naver_income_stmt_cum

,계정,2024-3Q,2024-2Q,2024-1Q,2023,2023-3Q
0,DX,134357500000000,89363600000000,47292700000000,169992300000000,130444100000000
1,DS,80965200000000,51693800000000,23137300000000,66594500000000,44902100000000
2,SDC,21031700000000,13032600000000,5386400000000,30975400000000,21315800000000
3,Harman,10348900000000,6819300000000,3200300000000,14388500000000,10464100000000
4,영업수익,225082634000000,74068302000000,71915601000000,258935494000000,191155556000000
5,영업비용,198849376000000,63624424000000,65309592000000,252368518000000,187413297000000
6,영업이익,26233258000000,10443878000000,6606009000000,6566976000000,3742259000000
7,영업이익률,0,0,0,0,0
8,순이익,26696957000000,9841345000000,6754708000000,15487100000000,5844171000000
9,순이익률,0,0,0,0,0


In [6]:
# 사업부 추출
segments = []
for item in naver_income_stmt_cum.iloc[:, 0]:
    if item == '영업수익':
        break
    segments.append(item)

segments

['DX', 'DS', 'SDC', 'Harman']

In [7]:
present_quarter = naver_income_stmt_cum.iloc[:,1]
year_ago_quarter = naver_income_stmt_cum.iloc[:,1+4]
yoy = ((present_quarter - year_ago_quarter) / year_ago_quarter) * 100
yoy

0      3.000059
1     80.314952
2     -1.332814
3     -1.100907
4     17.748413
5      6.102064
6    601.000599
7           NaN
8    356.813413
9           NaN
dtype: float64

In [8]:
# YoY 계산 ((현재 - 이전) / 이전) * 100
present_quarter = naver_income_stmt_cum.iloc[:,1]
year_ago_quarter = naver_income_stmt_cum.iloc[:,1+4]

yoy = ((present_quarter - year_ago_quarter) / year_ago_quarter) * 100

# 각 사업부별 YoY와 사업부 이름을 함께 보여주기
for segment, growth in zip(naver_income_stmt_cum['계정'], yoy):
    if segment in segments:  # segments 리스트에 있는 사업부만 출력
        state.segment[segment] = {"yoy":growth}

state.segment

{'DX': {'yoy': 3.000059029116687},
 'DS': {'yoy': 80.31495186193965},
 'SDC': {'yoy': -1.3328141566349845},
 'Harman': {'yoy': -1.1009069102932885}}

In [9]:
# 영업수익 yoy 계산
for segment, growth in zip(naver_income_stmt_cum['계정'], yoy):
    if segment == '영업수익':  # segments 리스트에 있는 사업부만 출력
        state.segment['영업수익'] = {"yoy":growth}

state.segment

{'DX': {'yoy': 3.000059029116687},
 'DS': {'yoy': 80.31495186193965},
 'SDC': {'yoy': -1.3328141566349845},
 'Harman': {'yoy': -1.1009069102932885},
 '영업수익': {'yoy': 17.748413234716548}}

In [10]:
# 사업부별 매출액 추출
for segment, sales in zip(naver_income_stmt_cum['계정'], present_quarter):
    if segment in segments:
        state.segment[segment].update({"sales":sales})

state.segment

{'DX': {'yoy': 3.000059029116687, 'sales': 134357500000000},
 'DS': {'yoy': 80.31495186193965, 'sales': 80965200000000},
 'SDC': {'yoy': -1.3328141566349845, 'sales': 21031700000000},
 'Harman': {'yoy': -1.1009069102932885, 'sales': 10348900000000},
 '영업수익': {'yoy': 17.748413234716548}}

In [11]:
# 영업수익 추출
for segment, sales in zip(naver_income_stmt_cum['계정'], present_quarter):
    if segment == '영업수익':  
        state.segment['영업수익'].update({"sales":sales})

state.segment

{'DX': {'yoy': 3.000059029116687, 'sales': 134357500000000},
 'DS': {'yoy': 80.31495186193965, 'sales': 80965200000000},
 'SDC': {'yoy': -1.3328141566349845, 'sales': 21031700000000},
 'Harman': {'yoy': -1.1009069102932885, 'sales': 10348900000000},
 '영업수익': {'yoy': 17.748413234716548, 'sales': 225082634000000}}

In [12]:
duckduckgo_search_tool = DuckDuckGoSearchResults(max_results=1,backend="news")

In [13]:
import time
for segment in segments:
    news_result = duckduckgo_search_tool.invoke(
        {
            "query": f"{state.company_name} {segment} 사업부 매출"
        }
    )
    state.segment[segment].update({"news_result":news_result})
    
state.segment

{'DX': {'yoy': 3.000059029116687,
  'sales': 134357500000000,
  'news_result': "snippet: 삼성전자는 전경훈 사장을 포함해 전기·전자공학 분야에서 최고로 인정받는 5명의 IEEE 펠로우를 동시에 배출했다. IEEE 펠로우는 IEEE 회원 중 최상위 0.1% 이내로 선정되는 최고 기술자 등급이다., title: 삼성전자, 전경훈 사장 등 5명 美 IEEE 펠로우 선정, link: https://www.msn.com/ko-kr/news/other/삼성전자-전경훈-사장-등-5명-美-ieee-펠로우-선정/ar-AA1vYXR0, date: 2024-12-17T01:41:00+00:00, source: 이투데이 on MSN.com, snippet: [디지털데일리 옥송이 기자] 삼성전자가 인사 및 조직 개편을 마무리 지으면서 내년 업무 채비를 갖췄다. 모바일 및 가전 등 제품을 모두 아우르는 DX(디바이스 경험)부문은 한종희 대표이사를 ..., title: '한종희' 삼성전자 DX, 내년 키워드 '안정'…영업익 개선 숙제, link: https://www.ddaily.co.kr/page/view/2024120515115760549, date: 2024-12-05T16:26:00+00:00, source: 디지털데일리, snippet: [더팩트ㅣ이성락 기자] 위기론에 휩싸인 삼성전자가 주요 반도체 사업 수장을 교체하는 쇄신 인사를 단행했다. 삼성전자는 27일 사장 승진 2명, 위촉 업무 변경 7명 등 총 9명 규모의 2025년 정기 사장단 인사를 발표했다. 먼저 삼성전자 반도체(DS)부문 미주(DSA)총괄 한진만 부사장은 사장으로 승진, DS부문 파운드리사업부장을 맡게 됐다. 삼성전자 사, title: 삼성전자, 반도체 사업부장 교체 '쇄신 인사'…전영현 역할 확대, link: https://www.msn.com/ko-kr/money/경영진-리더십-및-관리/삼성전자-반도체-사업부장-교체-쇄신-인사-전영현-역

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class yoy_prediction(BaseModel):
    """yoy 예측값을 나타내는 모델"""
    business_segment: str = Field(description="사업부")
    yoy: float = Field(description="사업부 yoy 예측값")
    reason: str = Field(description="사업부 yoy 예측값의 근거")

parser = PydanticOutputParser(pydantic_object=yoy_prediction)


template = """
당신은 증권사 소속 애널리스트입니다.
다음 내용을 참고하여 {company_name}의 {business_segment} 사업부의 매출 혹은 비용이 yoy로 얼마나 변할지 예측하시오.

직전 분기 yoy : {yoy}

{company_name}의 {business_segment} 사업부의 매출 혹은 비용에 큰 영향을 미치는 뉴스 : {news}

응답 형식:
{format_instructions}
"""

for segment in segments:
    prompt = ChatPromptTemplate.from_template(template=template, partial_variables={"format_instructions": parser.get_format_instructions()})

    llm = ChatOpenAI(temperature=0, model="gpt-4o")

    chain = prompt | llm | parser

    result = chain.invoke({"company_name": state.company_name, "news": state.segment[segment]['news_result'], "business_segment": segment, "yoy": state.segment[segment]['yoy']})

    # 예측값 및 예측 이유업데이트
    state.segment[segment].update({"yoy_prediction":result.yoy})
    state.segment[segment].update({"yoy_prediction_reason":result.reason})

In [15]:
state.segment

{'DX': {'yoy': 3.000059029116687,
  'sales': 134357500000000,
  'news_result': "snippet: 삼성전자는 전경훈 사장을 포함해 전기·전자공학 분야에서 최고로 인정받는 5명의 IEEE 펠로우를 동시에 배출했다. IEEE 펠로우는 IEEE 회원 중 최상위 0.1% 이내로 선정되는 최고 기술자 등급이다., title: 삼성전자, 전경훈 사장 등 5명 美 IEEE 펠로우 선정, link: https://www.msn.com/ko-kr/news/other/삼성전자-전경훈-사장-등-5명-美-ieee-펠로우-선정/ar-AA1vYXR0, date: 2024-12-17T01:41:00+00:00, source: 이투데이 on MSN.com, snippet: [디지털데일리 옥송이 기자] 삼성전자가 인사 및 조직 개편을 마무리 지으면서 내년 업무 채비를 갖췄다. 모바일 및 가전 등 제품을 모두 아우르는 DX(디바이스 경험)부문은 한종희 대표이사를 ..., title: '한종희' 삼성전자 DX, 내년 키워드 '안정'…영업익 개선 숙제, link: https://www.ddaily.co.kr/page/view/2024120515115760549, date: 2024-12-05T16:26:00+00:00, source: 디지털데일리, snippet: [더팩트ㅣ이성락 기자] 위기론에 휩싸인 삼성전자가 주요 반도체 사업 수장을 교체하는 쇄신 인사를 단행했다. 삼성전자는 27일 사장 승진 2명, 위촉 업무 변경 7명 등 총 9명 규모의 2025년 정기 사장단 인사를 발표했다. 먼저 삼성전자 반도체(DS)부문 미주(DSA)총괄 한진만 부사장은 사장으로 승진, DS부문 파운드리사업부장을 맡게 됐다. 삼성전자 사, title: 삼성전자, 반도체 사업부장 교체 '쇄신 인사'…전영현 역할 확대, link: https://www.msn.com/ko-kr/money/경영진-리더십-및-관리/삼성전자-반도체-사업부장-교체-쇄신-인사-전영현-역

In [16]:
naver_income_stmt_cum

,계정,2024-3Q,2024-2Q,2024-1Q,2023,2023-3Q
0,DX,134357500000000,89363600000000,47292700000000,169992300000000,130444100000000
1,DS,80965200000000,51693800000000,23137300000000,66594500000000,44902100000000
2,SDC,21031700000000,13032600000000,5386400000000,30975400000000,21315800000000
3,Harman,10348900000000,6819300000000,3200300000000,14388500000000,10464100000000
4,영업수익,225082634000000,74068302000000,71915601000000,258935494000000,191155556000000
5,영업비용,198849376000000,63624424000000,65309592000000,252368518000000,187413297000000
6,영업이익,26233258000000,10443878000000,6606009000000,6566976000000,3742259000000
7,영업이익률,0,0,0,0,0
8,순이익,26696957000000,9841345000000,6754708000000,15487100000000,5844171000000
9,순이익률,0,0,0,0,0


In [17]:
# 다음 분기(예측) 열 추가 (모든 행 0으로 초기화)
naver_income_stmt_cum['next_quarter'] = 0

# segments에 있는 계정에 대해서만 yoy_prediction을 적용하여 업데이트
for segment in segments:
    yoy_prediction = state.segment[segment]['yoy_prediction']
    growth_rate = 1 + (yoy_prediction / 100)  # yoy_prediction을 성장률로 변환
    
    # 해당 segment 행의 next_quarter 값을 업데이트(next_quarter의 1년 전 분기 값 * 성장률)
    naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == segment, 'next_quarter'] = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == segment].iloc[:,4] * growth_rate

naver_income_stmt_cum

,계정,2024-3Q,2024-2Q,2024-1Q,2023,2023-3Q,next_quarter
0,DX,134357500000000,89363600000000,47292700000000,169992300000000,130444100000000,175177065150000
1,DS,80965200000000,51693800000000,23137300000000,66594500000000,44902100000000,123199825000000
2,SDC,21031700000000,13032600000000,5386400000000,30975400000000,21315800000000,30990887700000
3,Harman,10348900000000,6819300000000,3200300000000,14388500000000,10464100000000,14237420750000
4,영업수익,225082634000000,74068302000000,71915601000000,258935494000000,191155556000000,0
5,영업비용,198849376000000,63624424000000,65309592000000,252368518000000,187413297000000,0
6,영업이익,26233258000000,10443878000000,6606009000000,6566976000000,3742259000000,0
7,영업이익률,0,0,0,0,0,0
8,순이익,26696957000000,9841345000000,6754708000000,15487100000000,5844171000000,0
9,순이익률,0,0,0,0,0,0


In [ ]:
# next_quarter의 segments 행들의 합계 계산
revenue_sum = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'].isin(segments), 'next_quarter'].sum()

# segments의 합계를 영업수익 값으로 업데이트
naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익', 'next_quarter'] = revenue_sum
naver_income_stmt_cum

In [ ]:
# 직전분기의 영업수익 대비 영업비용 비율 계산
revenue_current_quarter = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익'].iloc[:,1].values[0]
cost_current_quarter = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업비용'].iloc[:,1].values[0]
cost_ratio = cost_current_quarter / revenue_current_quarter

# 동기간 전분기의 영업비용비율을 다음 분기의 영업비용 예측에 사용(영업이익률을 그대로 사용)
revenue_next_quarter = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익', 'next_quarter'].values[0]
cost_next_quarter = revenue_next_quarter * cost_ratio

# 영업비용 행의 next_quarter 값을 업데이트
naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업비용', 'next_quarter'] = cost_next_quarter

naver_income_stmt_cum

In [ ]:
# next_quarter의 영업이익 계산 (영업수익 - 영업비용)
operating_profit_next_quarter = revenue_next_quarter - cost_next_quarter

# 영업이익 행의 next_quarter 값을 업데이트
naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업이익', 'next_quarter'] = operating_profit_next_quarter

naver_income_stmt_cum

In [ ]:
# 다음분기의 영업이익률 계산 (영업이익/영업수익 * 100)
operating_margin_next_quarter = (operating_profit_next_quarter / revenue_next_quarter)

# 영업이익률 행의 next_quarter 값을 업데이트
naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업이익률', 'next_quarter'] = operating_margin_next_quarter

naver_income_stmt_cum

In [ ]:
# 직전분기의 순이익률 계산
net_income_current_quarter = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '순이익'].iloc[:,1].values[0]
net_margin_current_quarter = net_income_current_quarter / revenue_current_quarter

# 다음분기의 순이익 계산(직전분기의 순이익률 * 다음분기의 추정 영업수익)
net_income_next_quarter = revenue_next_quarter * net_margin_current_quarter

# 순이익과 순이익률 행 업데이트
naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '순이익', 'next_quarter'] = net_income_next_quarter
naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '순이익률', 'next_quarter'] = net_margin_current_quarter

naver_income_stmt_cum

In [23]:
def naver_sales_earning_surprise() -> float:
    """네이버 증권 -> 종목분석 -> 컨센서스 -> 어닝서프라이즈(매출액) -> 분기
        return :
        직전 분기 매출액에 대한 발표직전 컨센서스 (단위: 억원) + 직전 분기의 전분기까지의 실제 매출액 = 직전 분기까지 누적 매출액 컨센서스
    """
    current_quarter_sales_consensus = 26620.4 * (10 ** 8) # 크롤링 추가 필요
    before_current_quarter_sales_cum = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익'].iloc[:,2].values[0]

    return current_quarter_sales_consensus + before_current_quarter_sales_cum


In [ ]:
current_quarter_sales_cum_consensus = naver_sales_earning_surprise()
current_quarter_sales_cum_consensus

In [ ]:
current_quarter_sales_cum = state.segment['영업수익']['sales']
current_quarter_sales_cum

In [ ]:
year_ago_quarter_sales_cum = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '영업수익'].iloc[:,5].values[0]
yoy_consensus = ((current_quarter_sales_cum_consensus - year_ago_quarter_sales_cum) / year_ago_quarter_sales_cum) * 100
yoy_consensus

In [27]:
state.segment['영업수익'].update({"yoy_consensus":yoy_consensus})
state.segment['영업수익'].update({"sales_consensus":current_quarter_sales_cum_consensus})

In [ ]:
state.segment['영업수익']

In [29]:
def current_quarter_review(state:State) -> None:
    """직전 분기 실적 리뷰를 작성"""

    result_of_business_segment = ""
    for segment in segments:
        result_of_business_segment += f"{segment} 사업부의 직전 분기 매출액은 {state.segment[segment]['sales']} 입니다. yoy로 {state.segment[segment]['yoy']}% 변했습니다.\n"

    result_of_total_business = f"총 매출액은 {state.segment['영업수익']['sales']} 입니다. yoy로 {state.segment['영업수익']['yoy']}% 변했습니다."

    consensus_of_total_business = f"총 매출액에 대한 컨센서스는 {state.segment['영업수익']['sales_consensus']} 였습니다. yoy로 {state.segment['영업수익']['yoy_consensus']}% 변화하는 것이 컨센서스였습니다."

    template = """
    당신은 증권사 소속 애널리스트입니다.
    다음 내용을 참고하여 직전 분기 실적에 대한 리뷰를 작성하시오.

    직전 분기 사업부별 매출 실적 : {result_of_business_segment}
    직전 분기 총 매출 실적 : {result_of_total_business}
    직전 분기 총 매출 실적에 대한 컨센서스 : {consensus_of_total_business}

    """
    prompt = ChatPromptTemplate.from_template(template=template)
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    chain = prompt | llm 
    review = chain.invoke({"result_of_business_segment":result_of_business_segment, "result_of_total_business":result_of_total_business, "consensus_of_total_business":consensus_of_total_business})
    state.segment['영업수익'].update({"review":review.content})

    return None

current_quarter_review(state)

In [ ]:
print(state.segment['영업수익']['review'])

In [32]:
from yahooquery import search
from deep_translator import GoogleTranslator

# 한글 문자 범위를 이용해 한글 포함 여부 확인
def contains_korean(text):
    for char in text:
        if '\uac00' <= char <= '\ud7a3' or '\u3131' <= char <= '\u318e':
            return True
    return False

def get_ticker(company_name):
    try:            
        # 한글 포함 여부 확인
        is_korean = contains_korean(company_name)
        if is_korean:
            # 회사명을 영어로 번역
            translated = GoogleTranslator(source='auto', target='en').translate(company_name)
            
            # 번역된 이름으로 검색
            results = search(translated)
        else:
            results = search(company_name)
            
        # KSC, NYSE, NASDAQ, AMEX, JPX, HKG 순서로 찾기
        for quote in results['quotes']:
            if quote['exchange'] == 'KSC': # 한국
                return quote['symbol']
            elif quote['exchange'] == 'NYQ': # NYSE
                return quote['symbol']
            elif quote['exchange'] == 'NMS': # NASDAQ
                return quote['symbol']
            elif quote['exchange'] == 'JPX': # 일본
                return quote['symbol']
            elif quote['exchange'] == 'HKG': # 홍콩
                return quote['symbol']
            else:
                continue
        
        # KSC, NYSE, NASDAQ, AMEX에 없으면 None 반환
        return None
    except Exception as e:
        print(f"Error translating or searching for {company_name}: {e}")
        return None

In [33]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate

response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변, 파이썬 리스트 형식이어야 함."),
    ]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

In [34]:
import yfinance as yf
import numpy as np

def find_peer(company: str) -> list[str]:
    prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
    )
    chain = prompt | llm | output_parser  # 프롬프트, 모델, 출력 파서를 연결
    peer_list = chain.invoke({"question": f"{company}와 사업구조가 비슷하고, 같은 산업 혹은 섹터에 속한 경쟁사는?"
                              "(코스피, 뉴욕거래소 등 상장된 회사만 찾으세요. 반드시 회사명만 출력해주세요.)"})
    return peer_list

def find_peer_PERs_tool(company: str) -> None:
    """기업과 동종 업계의 Peer Group PER 평균을 찾습니다."""
    ticker = get_ticker(company)
    peer_list = find_peer(company)['answer']
    if ticker is None:
        return None
    
    peer_pers = {}
    for peer in peer_list:
        ticker = get_ticker(peer)
        if ticker is None:
            continue
        elif ".KS" in ticker:
            ticker = yf.Ticker(ticker)
            earning_ttm = 0
            for i in range(4):
                earning_ttm += ticker.quarterly_income_stmt.loc['Net Income Common Stockholders'][i]
            trailingPERttm = ticker.info.get("marketCap")/earning_ttm
            if trailingPERttm <0 :
                continue
            peer_pers[peer] = trailingPERttm
        else:
            ticker = yf.Ticker(ticker)
            earning_ttm = 0
            for i in range(4):
                earning_ttm += ticker.quarterly_income_stmt.loc['Net Income Common Stockholders'][i]
            trailingPERttm = ticker.info.get("marketCap")/earning_ttm*0.7 # 외국 주식의 경우 PER을 30% 할인
            if trailingPERttm <0 :
                continue
            if np.isnan(trailingPERttm):
                continue
            peer_pers[peer] = trailingPERttm

    print(peer_pers)

    valid_peer_pers = {key: value for key, value in peer_pers.items() 
                 if 0.5 * state.PER <= value <= 2 * state.PER}
    
    print(valid_peer_pers)
    
    average_peer_per = sum(valid_peer_pers.values()) / len(valid_peer_pers)

    state.peer_list = peer_list
    state.average_peer_PER = average_peer_per

    return None

In [35]:
def find_PER_tool(company: str) -> None:
    """기업의 현재 PER(TTM)를 찾습니다."""
    ticker = get_ticker(company)
    if ticker is None:
        return None
    else:
        ticker = yf.Ticker(ticker)
        earning_ttm = 0
        for i in range(4):
            earning_ttm += ticker.quarterly_income_stmt.loc['Net Income Common Stockholders'][i]
        trailingPERttm = ticker.info["marketCap"]/earning_ttm
        state.PER = trailingPERttm
        return None


In [ ]:
find_PER_tool(state.company_name)
find_peer_PERs_tool(state.company_name)

In [ ]:
state.PER

In [ ]:
state.peer_list

In [ ]:
state.average_peer_PER

In [45]:
previous_quarter_review_result = state.segment['영업수익']['review']
segment_yoy_prediction_result = ""
for segment in segments:
    segment_yoy_prediction_result += f"다음 분기 {segment} 사업부의 매출액은 yoy로 {state.segment[segment]['yoy_prediction']}% 변화할 것으로 예측됩니다.\n 직전 분기 {segment} 사업부의 매출액은 {state.segment[segment]['yoy']}% 변화했습니다.\n"
    segment_yoy_prediction_result += f"그 이유는 다음과 같습니다. {state.segment[segment]['yoy_prediction_reason']}\n"
    segment_yoy_prediction_result += f"이 예측은 다음과 같은 뉴스를 기반으로 이루어졌습니다. {state.segment[segment]['news_result']}\n"

template = """
당신은 증권사 소속 애널리스트입니다.
다음 내용을 참고하여 각 사업부별 매출액 예측결과와 근거를 사업부별로 정리하여 작성하세요.
근거를 서술할 때에는 근거가 된 뉴스를 반드시 언급하세요.
다음 분기 매출액 yoy 증가율 예측결과와 직전분기 매출액 yoy 증가율을 비교하여 예측결과가 더 정확할 것으로 예상되는 이유를 서술하세요.

사업부별 매출 예측 결과와 근거 : {segment_yoy_prediction_result}
"""
prompt = ChatPromptTemplate.from_template(template=template)
llm = ChatOpenAI(temperature=0, model="gpt-4o")
chain = prompt | llm
segment_yoy_prediction_result = chain.invoke({"segment_yoy_prediction_result": segment_yoy_prediction_result}).content

In [ ]:
print(segment_yoy_prediction_result)

In [47]:
ticker = get_ticker(state.company_name)
ticker = yf.Ticker(ticker)
estEarnings = naver_income_stmt_cum.loc[naver_income_stmt_cum['계정'] == '순이익','next_quarter'].values[0]
shares = ticker.info.get("sharesOutstanding")
EPS = estEarnings/shares

In [ ]:
estEarnings

In [ ]:
EPS

In [50]:
price_consensus = ticker.analyst_price_targets

In [ ]:
price_consensus['mean']

In [52]:
price_consensus = ticker.analyst_price_targets

template = """
당신은 증권사 소속 애널리스트입니다.
다음 내용을 참고하여 {company_name}의 목표 주가를 설정하고 그 이유를 반드시 서술하세요.
직전분기 매출 실적 리뷰 내용과 사업부별 매출 예측 결과와 근거는 반드시 서술하세요.
목표 주가를 설정할 때에는 먼저, 현재 기업 PER과 경쟁사들의 평균 PER을 반드시 고려하여 기업의 목표 PER을 설정하세요.
목표 PER과 추정 EPS를 곱하여 목표 주가를 설정하세요.

직전 분기 매출 실적 리뷰: {previous_quarter_review_result}

사업부별 매출 예측 결과와 근거 : {segment_yoy_prediction_result}

현재 기업 PER : {PER}

기업 경쟁사 리스트 : {peer_list}

현재 경쟁사들의 평균 PER : {average_peer_PER}

추정 EPS : {EPS} (추정 매출액을 기반으로 계산된 결과입니다.)

목표 주가를 제안한 이후, 목표 주가 컨센서스를 아래 내용을 참고하여 작성하세요.

평균 목표 주가 : {avg_price_consensus}
최소 목표 주가 : {low_price_consensus}
최대 목표 주가 : {high_price_consensus}
현재 주가 : {current_price}
"""


prompt = ChatPromptTemplate.from_template(template=template)
llm = ChatOpenAI(temperature=0, model="gpt-4o")
chain = prompt | llm
valuation_result = chain.invoke({"company_name": "네이버", 
              "previous_quarter_review_result": previous_quarter_review_result, 
              "segment_yoy_prediction_result": segment_yoy_prediction_result, 
              "PER": state.PER, 
              "average_peer_PER": state.average_peer_PER,
              "peer_list": state.peer_list,
              "EPS": EPS,
              "avg_price_consensus": price_consensus['mean'],
              "low_price_consensus": price_consensus['low'],
              "high_price_consensus": price_consensus['high'],
              "current_price": ticker.info.get("regularMarketPrice")})

In [ ]:
print(valuation_result.content)